# 📊 Analyse Univariée Détaillée
## Projet: Détection de Fraudes dans les Transactions Bancaires

**Auteur:** Dady Akrou Cyrille  
**Email:** cyrilledady0501@gmail.com  
**Institution:** UQTR - Université du Québec à Trois-Rivières  
**Dataset:** Credit Card Fraud Detection (Kaggle)

---

## Objectifs de cette phase:
1. Analyser en détail chaque variable individuellement
2. Comparer les distributions entre fraudes et transactions normales
3. Identifier les variables les plus discriminantes
4. Détecter les outliers et anomalies
5. Préparer les insights pour l'analyse multivariée

In [ ]:
# Import des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (15, 8)

print('✅ Librairies importées avec succès!')

In [ ]:
# Chargement des données
df = pd.read_csv('../Credit Card Fraud Detection/creditcard.csv')

# Séparation des données par classe
fraud = df[df['Class'] == 1]
normal = df[df['Class'] == 0]

print(f'📊 Dataset chargé: {df.shape}')
print(f'🚨 Fraudes: {len(fraud):,} transactions')
print(f'✅ Normales: {len(normal):,} transactions')

## 1. Analyse de la Variable Time

In [ ]:
# Analyse temporelle
print('⏰ ANALYSE DE LA VARIABLE TIME')
print('='*40)

# Conversion en heures pour une meilleure lisibilité
df['Time_hours'] = df['Time'] / 3600
fraud['Time_hours'] = fraud['Time'] / 3600
normal['Time_hours'] = normal['Time'] / 3600

# Statistiques par classe
print('📈 Statistiques temporelles par classe:')
time_stats = pd.DataFrame({
    'Normal': normal['Time_hours'].describe(),
    'Fraude': fraud['Time_hours'].describe()
})
display(time_stats.round(2))

# Test statistique
stat, p_value = stats.mannwhitneyu(normal['Time_hours'], fraud['Time_hours'])
print(f'\n🧪 Test Mann-Whitney U:')
print(f'   Statistique: {stat:.2f}')
print(f'   P-value: {p_value:.2e}')
print(f'   Différence significative: {"Oui" if p_value < 0.05 else "Non"}')

In [ ]:
# Visualisation de la distribution temporelle
fig, axes = plt.subplots(2, 2, figsize=(20, 12))

# Distribution générale
axes[0,0].hist(df['Time_hours'], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
axes[0,0].set_title('Distribution Temporelle Générale', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Temps (heures)')
axes[0,0].set_ylabel('Nombre de Transactions')
axes[0,0].grid(True, alpha=0.3)

# Comparaison par classe
axes[0,1].hist(normal['Time_hours'], bins=50, alpha=0.6, label='Normal', color='lightblue')
axes[0,1].hist(fraud['Time_hours'], bins=50, alpha=0.8, label='Fraude', color='red')
axes[0,1].set_title('Distribution Temporelle par Classe', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Temps (heures)')
axes[0,1].set_ylabel('Nombre de Transactions')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Box plot comparatif
data_box = [normal['Time_hours'], fraud['Time_hours']]
axes[1,0].boxplot(data_box, labels=['Normal', 'Fraude'], patch_artist=True,
                  boxprops=dict(facecolor='lightblue'),
                  medianprops=dict(color='red', linewidth=2))
axes[1,0].set_title('Box Plot - Distribution Temporelle', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('Temps (heures)')
axes[1,0].grid(True, alpha=0.3)

# Analyse par période de la journée
df['Hour_of_day'] = (df['Time_hours'] % 24).astype(int)
fraud_by_hour = df[df['Class']==1]['Hour_of_day'].value_counts().sort_index()
normal_by_hour = df[df['Class']==0]['Hour_of_day'].value_counts().sort_index()

axes[1,1].plot(fraud_by_hour.index, fraud_by_hour.values, 'ro-', label='Fraudes', linewidth=2)
axes[1,1].plot(normal_by_hour.index, normal_by_hour.values/100, 'bo-', label='Normales (/100)', linewidth=2)
axes[1,1].set_title('Transactions par Heure de la Journée', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Heure')
axes[1,1].set_ylabel('Nombre de Transactions')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)
axes[1,1].set_xticks(range(0, 24, 2))

plt.tight_layout()
plt.show()

# Sauvegarde
plt.savefig('../reports/figures/02_analyse_temporelle.png', dpi=300, bbox_inches='tight')
print('💾 Graphique sauvegardé: reports/figures/02_analyse_temporelle.png')

## 2. Analyse de la Variable Amount

In [ ]:
# Analyse des montants
print('💰 ANALYSE DE LA VARIABLE AMOUNT')
print('='*40)

# Statistiques par classe
amount_stats = pd.DataFrame({
    'Normal': normal['Amount'].describe(),
    'Fraude': fraud['Amount'].describe()
})
display(amount_stats.round(2))

# Test statistique
stat, p_value = stats.mannwhitneyu(normal['Amount'], fraud['Amount'])
print(f'\n🧪 Test Mann-Whitney U:')
print(f'   Statistique: {stat:.2f}')
print(f'   P-value: {p_value:.2e}')
print(f'   Différence significative: {"Oui" if p_value < 0.05 else "Non"}')

# Analyse des percentiles
print('\n📊 Analyse des percentiles:')
percentiles = [25, 50, 75, 90, 95, 99]
for p in percentiles:
    normal_p = np.percentile(normal['Amount'], p)
    fraud_p = np.percentile(fraud['Amount'], p)
    print(f'   P{p}: Normal=${normal_p:.2f}, Fraude=${fraud_p:.2f}')

In [ ]:
# Visualisation des montants
fig, axes = plt.subplots(2, 3, figsize=(22, 12))

# Distribution générale
axes[0,0].hist(df['Amount'], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
axes[0,0].set_title('Distribution des Montants (Générale)', fontsize=12, fontweight='bold')
axes[0,0].set_xlabel('Montant ($)')
axes[0,0].set_ylabel('Fréquence')
axes[0,0].grid(True, alpha=0.3)

# Distribution log-scale
amount_nonzero = df[df['Amount'] > 0]['Amount']
axes[0,1].hist(np.log10(amount_nonzero), bins=50, alpha=0.7, color='green', edgecolor='black')
axes[0,1].set_title('Distribution des Montants (Log Scale)', fontsize=12, fontweight='bold')
axes[0,1].set_xlabel('Log10(Montant)')
axes[0,1].set_ylabel('Fréquence')
axes[0,1].grid(True, alpha=0.3)

# Comparaison par classe
axes[0,2].hist(normal['Amount'], bins=50, alpha=0.6, label='Normal', color='lightblue', density=True)
axes[0,2].hist(fraud['Amount'], bins=50, alpha=0.8, label='Fraude', color='red', density=True)
axes[0,2].set_title('Distribution des Montants par Classe', fontsize=12, fontweight='bold')
axes[0,2].set_xlabel('Montant ($)')
axes[0,2].set_ylabel('Densité')
axes[0,2].legend()
axes[0,2].grid(True, alpha=0.3)

# Box plot
data_box = [normal['Amount'], fraud['Amount']]
box_plot = axes[1,0].boxplot(data_box, labels=['Normal', 'Fraude'], patch_artist=True,
                             boxprops=dict(facecolor='lightblue'),
                             medianprops=dict(color='red', linewidth=2))
axes[1,0].set_title('Box Plot - Montants par Classe', fontsize=12, fontweight='bold')
axes[1,0].set_ylabel('Montant ($)')
axes[1,0].grid(True, alpha=0.3)

# Violin plot
data_violin = pd.DataFrame({
    'Amount': np.concatenate([normal['Amount'], fraud['Amount']]),
    'Class': ['Normal']*len(normal) + ['Fraude']*len(fraud)
})
sns.violinplot(data=data_violin, x='Class', y='Amount', ax=axes[1,1])
axes[1,1].set_title('Violin Plot - Distribution des Montants', fontsize=12, fontweight='bold')
axes[1,1].grid(True, alpha=0.3)

# Analyse par tranches de montants
bins = [0, 10, 50, 100, 500, 1000, 5000, float('inf')]
labels = ['0-10$', '10-50$', '50-100$', '100-500$', '500-1K$', '1K-5K$', '5K+$']

df['Amount_range'] = pd.cut(df['Amount'], bins=bins, labels=labels, include_lowest=True)
fraud_by_range = df[df['Class']==1]['Amount_range'].value_counts()
normal_by_range = df[df['Class']==0]['Amount_range'].value_counts()

x_pos = np.arange(len(labels))
width = 0.35

axes[1,2].bar(x_pos - width/2, normal_by_range.reindex(labels, fill_value=0)/1000, 
              width, label='Normal (/1000)', color='lightblue', alpha=0.7)
axes[1,2].bar(x_pos + width/2, fraud_by_range.reindex(labels, fill_value=0), 
              width, label='Fraude', color='red', alpha=0.7)
axes[1,2].set_title('Transactions par Tranche de Montant', fontsize=12, fontweight='bold')
axes[1,2].set_xlabel('Tranche de Montant')
axes[1,2].set_ylabel('Nombre de Transactions')
axes[1,2].set_xticks(x_pos)
axes[1,2].set_xticklabels(labels, rotation=45)
axes[1,2].legend()
axes[1,2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Sauvegarde
plt.savefig('../reports/figures/02_analyse_montants.png', dpi=300, bbox_inches='tight')
print('💾 Graphique sauvegardé: reports/figures/02_analyse_montants.png')

## 3. Analyse des Variables PCA (V1-V28)

In [ ]:
# Analyse des variables PCA
pca_columns = [col for col in df.columns if col.startswith('V')]

print('🔢 ANALYSE DES VARIABLES PCA')
print('='*40)

# Calcul des statistiques pour chaque variable PCA
pca_analysis = []

for col in pca_columns:
    # Test statistique
    stat, p_value = stats.mannwhitneyu(normal[col], fraud[col])
    
    # Moyennes
    mean_normal = normal[col].mean()
    mean_fraud = fraud[col].mean()
    
    # Écarts-types
    std_normal = normal[col].std()
    std_fraud = fraud[col].std()
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(normal)-1)*std_normal**2 + (len(fraud)-1)*std_fraud**2) / (len(normal)+len(fraud)-2))
    cohens_d = abs(mean_fraud - mean_normal) / pooled_std
    
    pca_analysis.append({
        'Variable': col,
        'Mean_Normal': mean_normal,
        'Mean_Fraud': mean_fraud,
        'Std_Normal': std_normal,
        'Std_Fraud': std_fraud,
        'P_value': p_value,
        'Cohens_d': cohens_d,
        'Significant': p_value < 0.05
    })

pca_df = pd.DataFrame(pca_analysis)
pca_df = pca_df.sort_values('Cohens_d', ascending=False)

print('📊 Top 10 variables les plus discriminantes (Cohen\'s d):')
display(pca_df.head(10)[['Variable', 'Mean_Normal', 'Mean_Fraud', 'Cohens_d', 'Significant']].round(4))

print(f'\n🎯 Variables significatives: {pca_df["Significant"].sum()}/{len(pca_columns)}')

In [ ]:
# Visualisation des variables PCA les plus importantes
top_variables = pca_df.head(8)['Variable'].tolist()

fig, axes = plt.subplots(4, 4, figsize=(20, 16))
axes = axes.ravel()

for i, var in enumerate(top_variables):
    # Distribution par classe
    axes[i*2].hist(normal[var], bins=50, alpha=0.6, label='Normal', color='lightblue', density=True)
    axes[i*2].hist(fraud[var], bins=50, alpha=0.8, label='Fraude', color='red', density=True)
    axes[i*2].set_title(f'{var} - Distribution par Classe', fontsize=10, fontweight='bold')
    axes[i*2].set_xlabel(var)
    axes[i*2].set_ylabel('Densité')
    axes[i*2].legend()
    axes[i*2].grid(True, alpha=0.3)
    
    # Box plot
    data_box = [normal[var], fraud[var]]
    axes[i*2+1].boxplot(data_box, labels=['Normal', 'Fraude'], patch_artist=True,
                        boxprops=dict(facecolor='lightblue'),
                        medianprops=dict(color='red', linewidth=2))
    axes[i*2+1].set_title(f'{var} - Box Plot', fontsize=10, fontweight='bold')
    axes[i*2+1].grid(True, alpha=0.3)

plt.suptitle('Top 8 Variables PCA les Plus Discriminantes', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

# Sauvegarde
plt.savefig('../reports/figures/02_top_variables_pca.png', dpi=300, bbox_inches='tight')
print('💾 Graphique sauvegardé: reports/figures/02_top_variables_pca.png')

In [ ]:
# Heatmap des effect sizes
plt.figure(figsize=(15, 8))

# Préparation des données pour la heatmap
effect_sizes = pca_df.set_index('Variable')['Cohens_d'].values.reshape(1, -1)

# Création de la heatmap
sns.heatmap(effect_sizes, 
            xticklabels=pca_df['Variable'], 
            yticklabels=['Cohen\'s d'],
            annot=True, 
            fmt='.3f',
            cmap='YlOrRd',
            cbar_kws={'label': 'Effect Size (Cohen\'s d)'})

plt.title('Effect Size (Cohen\'s d) pour toutes les Variables PCA', fontsize=14, fontweight='bold')
plt.xlabel('Variables PCA')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Sauvegarde
plt.savefig('../reports/figures/02_effect_sizes_heatmap.png', dpi=300, bbox_inches='tight')
print('💾 Graphique sauvegardé: reports/figures/02_effect_sizes_heatmap.png')

## 4. Détection d'Outliers

In [ ]:
# Détection d'outliers avec la méthode IQR
print('🎯 DÉTECTION D\'OUTLIERS')
print('='*40)

def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analyse des outliers pour les variables principales
main_vars = ['Time', 'Amount'] + top_variables[:5]

outlier_summary = []

for var in main_vars:
    # Outliers dans les transactions normales
    outliers_normal, lb_n, ub_n = detect_outliers_iqr(normal, var)
    
    # Outliers dans les fraudes
    outliers_fraud, lb_f, ub_f = detect_outliers_iqr(fraud, var)
    
    outlier_summary.append({
        'Variable': var,
        'Outliers_Normal': len(outliers_normal),
        'Outliers_Fraud': len(outliers_fraud),
        'Pct_Normal': len(outliers_normal)/len(normal)*100,
        'Pct_Fraud': len(outliers_fraud)/len(fraud)*100
    })

outlier_df = pd.DataFrame(outlier_summary)
print('📊 Résumé des outliers par variable:')
display(outlier_df.round(2))

In [ ]:
# Visualisation des outliers
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, var in enumerate(['Time', 'Amount'] + top_variables[:4]):
    # Box plot avec outliers
    data_box = [normal[var], fraud[var]]
    box_plot = axes[i].boxplot(data_box, labels=['Normal', 'Fraude'], 
                               patch_artist=True, showfliers=True,
                               boxprops=dict(facecolor='lightblue', alpha=0.7),
                               medianprops=dict(color='red', linewidth=2),
                               flierprops=dict(marker='o', markerfacecolor='red', 
                                             markersize=3, alpha=0.5))
    
    axes[i].set_title(f'Outliers - {var}', fontsize=12, fontweight='bold')
    axes[i].set_ylabel(var)
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Détection d\'Outliers par Variable', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Sauvegarde
plt.savefig('../reports/figures/02_outliers_detection.png', dpi=300, bbox_inches='tight')
print('💾 Graphique sauvegardé: reports/figures/02_outliers_detection.png')

## 5. Résumé de l'Analyse Univariée

In [ ]:
# Résumé des insights
print('📋 RÉSUMÉ DE L\'ANALYSE UNIVARIÉE')
print('='*50)

print('⏰ VARIABLE TIME:')
print('   - Distribution uniforme sur 48h')
print('   - Pas de pattern temporel spécifique aux fraudes')
print('   - Fraudes réparties uniformément dans le temps')

print('💰 VARIABLE AMOUNT:')
print('   - Fraudes concentrées sur petits montants')
print('   - Médiane fraudes: 22$ vs 84$ (normales)')
print('   - 75% des fraudes < 77$')
print('   - Aucune fraude > 2,126$')

print('🔢 VARIABLES PCA:')
print(f'   - {pca_df["Significant"].sum()}/{len(pca_columns)} variables significatives')
print(f'   - Top 3: {"\n".join(pca_df.head(3)["Variable"].tolist())}')
print(f'   - Effect size max: {pca_df["Cohens_d"].max():.3f}')

print('🎯 OUTLIERS:')
print('   - Présence d\'outliers dans toutes les variables')
print('   - Fraudes souvent dans les valeurs extrêmes')
print('   - Nécessité de techniques robustes')

print('🚀 PROCHAINES ÉTAPES:')
print('   1. Analyse multivariée et corrélations')
print('   2. Réduction de dimensionnalité (PCA, t-SNE)')
print('   3. Clustering et segmentation')
print('   4. Feature engineering avancé')
print('   5. Modélisation et validation')